### features:
* selected spectogram coef
* zero crossing rate
* selected chromogram values
* selected tempogram values
* wavelet of level 0

In [1]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import auc
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, recall_score
import matplotlib.pyplot as plt
import itertools

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
csv_path = "/Users/ecem/Desktop/csv of scgs"
wavelet_csv = "/Users/ecem/Desktop/wavelet-csv"

## Load data

Here, our target is to predict disease of patients with multiple diseases. So; our y will be disease column of two column indicatin geach patients disease.

In [3]:
spec = pd.read_csv(csv_path + '/p-value-selected-intersection-multiple-dis-spec.csv', index_col = 0)
#spec  #uncomment to see dataframe

signal_proc = pd.read_csv(csv_path + '/multiple-disease-final-signal_processing.csv', index_col = 0)
y1 = signal_proc.disease1
y2 = signal_proc.disease2

zero_crossing = signal_proc.iloc[:,:21]
#zero_crossing #uncomment to see dataframe

tempogram = pd.read_csv(csv_path + '/p-value-selected-multiple-dis-temp.csv', index_col = 0)
#tempogram #uncomment to see dataframe

chromogram = pd.read_csv(csv_path + '/p-value-selected-multiple-dis-chromogram.csv', index_col = 0)
#chromogram #uncomment to see dataframe

w_lvl0 = pd.read_csv(csv_path + '/multiple-disease-wavelet-lvl0-coef.csv', index_col = 0).drop(['disease1', 'disease2'], axis =1)
w_lvl0 = w_lvl0.reset_index(drop=True)
#w_lvl0  #uncomment to see dataframe

This line will be used to encode the target value. It is not going to be used for other purposes.

In [4]:
signal_proc_t = pd.read_csv(csv_path + '/signal_processing.csv', index_col = 0)
labels= signal_proc_t.disease

In [5]:
features = pd.concat([spec, w_lvl0], axis = 1)
features = pd.concat([features, zero_crossing], axis = 1)
features = pd.concat([features, chromogram], axis = 1)
features = pd.concat([features, tempogram], axis = 1)

features

,freq_y1772,freq_z1773,freq_z1774,freq_x555,freq_y555,freq_z555,freq_x556,freq_y556,freq_z556,freq_x557,...,tempogram_y256,tempogram_z320,tempogram_y320,tempogram_x320,tempogram_z319,tempogram_y318,tempogram_z318,tempogram_x319,tempogram_y319,tempogram_z255
0,0.003383,0.002980,0.002855,0.002161,0.002094,0.002025,0.001971,0.001921,0.001869,0.001812,...,0.445113,1.978267e-09,2.062070e-08,9.183201e-08,2.766100e-07,0.000004,2.549855e-06,1.371246e-06,6.635903e-07,0.618049
1,0.004910,0.004310,0.004276,0.001875,0.002008,0.002132,0.002238,0.002317,0.002364,0.002378,...,0.538619,2.086170e-09,2.166218e-08,9.600761e-08,2.882793e-07,0.000005,2.655965e-06,1.426979e-06,6.905931e-07,0.654835
2,0.004735,0.004130,0.003788,0.002241,0.002232,0.002200,0.002142,0.002063,0.001966,0.001855,...,0.608095,2.098638e-09,2.191610e-08,9.778785e-08,2.947368e-07,0.000005,2.711798e-06,1.459085e-06,7.067081e-07,0.741303
3,0.004367,0.004345,0.004337,0.002082,0.002036,0.001993,0.001960,0.001941,0.001943,0.001987,...,0.715791,2.025586e-09,2.105369e-08,9.338860e-08,2.804590e-07,0.000004,2.585357e-06,1.388093e-06,6.717521e-07,0.788690
4,0.004148,0.003581,0.003203,0.001680,0.001666,0.001645,0.001618,0.001595,0.001595,0.001607,...,0.767121,1.932686e-09,2.006646e-08,8.889795e-08,2.668415e-07,0.000004,2.468796e-06,1.322954e-06,6.394051e-07,0.819613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,0.001908,0.002203,0.002238,0.002802,0.002739,0.002655,0.002559,0.002466,0.002393,0.002345,...,0.364195,2.059336e-09,2.124141e-08,9.317839e-08,2.766343e-07,0.000004,2.460065e-06,1.337205e-06,6.549272e-07,0.521096
323,0.001781,0.002088,0.002148,0.004011,0.003909,0.003803,0.003701,0.003584,0.003453,0.003323,...,0.277581,1.249031e-09,1.333401e-08,6.137947e-08,1.903869e-07,0.000003,1.851025e-06,9.828763e-07,4.674462e-07,0.671185
324,0.001544,0.001566,0.001578,0.002893,0.002796,0.002694,0.002628,0.002590,0.002553,0.002508,...,0.407483,1.720723e-09,1.811135e-08,8.183620e-08,2.500919e-07,0.000004,2.378340e-06,1.268592e-06,6.076013e-07,0.655393
325,0.000873,0.001025,0.001021,0.002950,0.002898,0.002809,0.002688,0.002544,0.002399,0.002275,...,0.270778,1.720426e-09,1.781081e-08,7.853572e-08,2.342196e-07,0.000004,2.101861e-06,1.139776e-06,5.565471e-07,0.476264


In [6]:
y = signal_proc.iloc[:,-2:].to_numpy()

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer
one_hot = MultiLabelBinarizer()

print(one_hot.fit_transform(y))

print(one_hot.classes_)

[[0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 ...
 [0 0 1 1]
 [0 0 1 1]
 [0 0 1 1]]
['AR' 'AS' 'MR' 'MS']


In [8]:
y = one_hot.fit_transform(y)

In [9]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold =  0.0001)  #  yüksek threshold, tek hastalıkta yaptığımız
_ = vt.fit(features)
masked = vt.get_support()
features_masked = features.loc[:, masked]
features_masked # uncomment to see the dataframe

,lvl0_x0,lvl0_x1,lvl0_x2,lvl0_x3,lvl0_x4,lvl0_x5,lvl0_x6,lvl0_x7,lvl0_x8,lvl0_x9,...,A♯_z,C_y,C♯_y,C♯_z,D_x,D_y,A_x,tempogram_x256,tempogram_y256,tempogram_z255
0,-0.244702,-0.236068,-0.226322,-0.194488,-0.021728,0.029453,0.076783,0.110819,0.253816,0.116260,...,0.898151,0.465531,0.514051,0.547413,0.656185,0.739712,0.655725,0.476022,0.445113,0.618049
1,0.254884,0.273069,0.271307,0.247969,0.263392,0.120253,0.045193,-0.057686,-0.051953,-0.003762,...,0.924117,0.473939,0.708674,0.700867,0.769493,0.803762,0.738146,0.553634,0.538619,0.654835
2,0.190560,0.230630,0.213007,0.147370,0.069320,0.135292,0.166964,0.162940,0.022627,-0.118658,...,0.923531,0.392634,0.630330,0.691809,0.833126,0.845480,0.728774,0.631292,0.608095,0.741303
3,0.167240,0.180006,0.173210,0.135965,0.063945,0.032931,0.065671,-0.073614,-0.033845,-0.015386,...,0.979937,0.410735,0.538759,0.552843,0.649974,0.709571,0.633180,0.721456,0.715791,0.788690
4,-0.115657,-0.116028,-0.118900,-0.146683,-0.236584,-0.242813,-0.284676,-0.106079,-0.087932,-0.004055,...,0.921828,0.488668,0.683789,0.608681,0.740227,0.805843,0.678758,0.767489,0.767121,0.819613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,-0.063624,-0.116311,-0.095682,0.029851,0.216770,0.126891,0.103904,0.139855,0.203691,0.152053,...,0.923160,0.492128,0.563289,0.620287,0.512644,0.655851,0.522895,0.383970,0.364195,0.521096
323,0.160955,0.127541,0.132432,0.181104,0.173737,0.177821,0.173468,0.133190,0.180795,0.164669,...,0.794955,0.486064,0.666815,0.681477,0.685774,0.721715,0.709370,0.424022,0.277581,0.671185
324,-0.183802,-0.172011,-0.168269,-0.157887,-0.052772,-0.094302,-0.033630,0.007438,0.066805,0.117408,...,0.978834,0.546043,0.702636,0.783930,0.685779,0.806930,0.751198,0.493365,0.407483,0.655393
325,0.201973,0.211014,0.219511,0.263884,0.466177,0.447585,0.487150,0.465016,0.473533,0.388000,...,0.947354,0.490822,0.596342,0.618947,0.578295,0.707697,0.723225,0.303590,0.270778,0.476264


In [10]:
X_train_, X_valid_, y_train_, y_valid_ = train_test_split(features_masked, y, train_size=0.8, test_size=0.2,
                                                                random_state=42)

In [11]:
import xgboost as xgb
classifier = MultiOutputClassifier(xgb.XGBClassifier(objective = 'binary:logistic'))

clf = Pipeline([('classify', classifier)])


In [12]:
clf.fit(X_train_, y_train_)


Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metric=None,
                                                               gamma=None,
                                                               gpu_id=None,
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None,
                                                               reg_alpha=None,
                                                               reg_lambda=None, ...)))])

In [13]:
yhat = clf.predict(X_valid_)

In [14]:
df_results_test = pd.DataFrame({'AR': y_valid_[:,0], 'AS': y_valid_[:,1], 'MR': y_valid_[:,2], 'MS': y_valid_[:,3],
                        'predicted AR': yhat[:,0],
                          'predicted AS': yhat[:,1],
                          'predicted MR': yhat[:,2], 
                          'predicted MS': yhat[:,3]})
df_results_test

,AR,AS,MR,MS,predicted AR,predicted AS,predicted MR,predicted MS
0,0,1,0,1,0,1,0,1
1,1,0,1,0,0,1,1,0
2,0,1,0,1,0,1,0,1
3,0,1,1,0,1,0,1,0
4,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...
61,0,1,0,1,0,1,0,1
62,0,1,1,0,1,0,1,0
63,1,0,1,0,0,0,1,0
64,0,1,0,1,1,1,0,0


In [15]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_results_test)

    AR  AS  MR  MS  predicted AR  predicted AS  predicted MR  predicted MS
0    0   1   0   1             0             1             0             1
1    1   0   1   0             0             1             1             0
2    0   1   0   1             0             1             0             1
3    0   1   1   0             1             0             1             0
4    1   0   1   0             0             1             1             0
5    0   1   0   1             0             1             0             1
6    0   1   0   1             0             1             0             1
7    1   1   0   0             1             1             0             0
8    1   1   0   0             1             1             0             0
9    1   0   1   0             0             1             1             0
10   1   1   0   0             1             1             0             0
11   0   0   1   1             0             0             1             1
12   0   0   1   1       

In [16]:
auc_y1 = roc_auc_score(y_valid_[:,0],yhat[:,0])
auc_y2 = roc_auc_score(y_valid_[:,1],yhat[:,1])
auc_y3 = roc_auc_score(y_valid_[:,2],yhat[:,2])
auc_y4 = roc_auc_score(y_valid_[:,3],yhat[:,3])

print("ROC AUC AR: %.4f, AS: %.4f, MR: %.4f, MS: %.4f " % (auc_y1, auc_y2, auc_y3, auc_y4))


ROC AUC AR: 0.4673, AS: 0.5005, MR: 0.9449, MS: 0.9318 


In [17]:
cm_y1 = confusion_matrix(y_valid_[:,0],yhat[:,0])
cm_y2 = confusion_matrix(y_valid_[:,1],yhat[:,1])
cm_y3 = confusion_matrix(y_valid_[:,2],yhat[:,2])
cm_y4 = confusion_matrix(y_valid_[:,3],yhat[:,3])
print(cm_y1)



[[22 18]
 [16 10]]


In [18]:
cm_y2

array([[11, 14],
       [18, 23]])

In [19]:
cm_y3

array([[21,  2],
       [ 1, 42]])

In [20]:
cm_y4

array([[44,  0],
       [ 3, 19]])

In [21]:
cr_y1 = classification_report(y_valid_[:,0],yhat[:,0]) # AR
cr_y2 = classification_report(y_valid_[:,1],yhat[:,1]) # AS
cr_y3 = classification_report(y_valid_[:,2],yhat[:,2]) #mr
cr_y4 = classification_report(y_valid_[:,3],yhat[:,3]) #MS

print (cr_y1)
print (cr_y2)
print (cr_y3)
print (cr_y4)




              precision    recall  f1-score   support

           0       0.58      0.55      0.56        40
           1       0.36      0.38      0.37        26

    accuracy                           0.48        66
   macro avg       0.47      0.47      0.47        66
weighted avg       0.49      0.48      0.49        66

              precision    recall  f1-score   support

           0       0.38      0.44      0.41        25
           1       0.62      0.56      0.59        41

    accuracy                           0.52        66
   macro avg       0.50      0.50      0.50        66
weighted avg       0.53      0.52      0.52        66

              precision    recall  f1-score   support

           0       0.95      0.91      0.93        23
           1       0.95      0.98      0.97        43

    accuracy                           0.95        66
   macro avg       0.95      0.94      0.95        66
weighted avg       0.95      0.95      0.95        66

              preci

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(features, y, train_size=0.8, test_size=0.2,
                                                                random_state=42)

In [23]:
classifier_f = MultiOutputClassifier(xgb.XGBClassifier(objective = 'binary:logistic'))

clf_f = Pipeline([('classify', classifier)])

clf_f.fit(X_train, y_train)

yhat_f = clf.predict(X_valid)

In [24]:
df_results_test_f = pd.DataFrame({'AR': y_valid[:,0], 'AS': y_valid[:,1], 'MR': y_valid[:,2], 'MS': y_valid[:,3],
                        'predicted AR': yhat_f[:,0],
                          'predicted AS': yhat_f[:,1],
                          'predicted MR': yhat_f[:,2], 
                          'predicted MS': yhat_f[:,3]})
df_results_test_f

,AR,AS,MR,MS,predicted AR,predicted AS,predicted MR,predicted MS
0,0,1,0,1,0,1,0,1
1,1,0,1,0,0,1,1,0
2,0,1,0,1,0,1,0,1
3,0,1,1,0,1,0,1,0
4,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...
61,0,1,0,1,0,1,0,1
62,0,1,1,0,1,0,1,0
63,1,0,1,0,0,1,1,0
64,0,1,0,1,0,1,0,1


In [25]:
auc_y1_ = roc_auc_score(y_valid[:,0],yhat_f[:,0])
auc_y2_ = roc_auc_score(y_valid[:,1],yhat_f[:,1])
auc_y3_ = roc_auc_score(y_valid[:,2],yhat_f[:,2])
auc_y4_ = roc_auc_score(y_valid[:,3],yhat_f[:,3])

print("ROC AUC AR: %.4f, AS: %.4f, MR: %.4f, MS: %.4f " % (auc_y1_, auc_y2_, auc_y3_, auc_y4_))

ROC AUC AR: 0.4990, AS: 0.4849, MR: 1.0000, MS: 1.0000 


In [26]:
cr_y1_ = classification_report(y_valid[:,0],yhat_f[:,0]) # AR
cr_y2_ = classification_report(y_valid[:,1],yhat_f[:,1]) # AS
cr_y3_ = classification_report(y_valid[:,2],yhat_f[:,2]) #mr
cr_y4_ = classification_report(y_valid[:,3],yhat_f[:,3]) #MS

print (cr_y1_)
print (cr_y2_)
print (cr_y3_)
print (cr_y4_)



              precision    recall  f1-score   support

           0       0.61      0.57      0.59        40
           1       0.39      0.42      0.41        26

    accuracy                           0.52        66
   macro avg       0.50      0.50      0.50        66
weighted avg       0.52      0.52      0.52        66

              precision    recall  f1-score   support

           0       0.36      0.36      0.36        25
           1       0.61      0.61      0.61        41

    accuracy                           0.52        66
   macro avg       0.48      0.48      0.48        66
weighted avg       0.52      0.52      0.52        66

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        43

    accuracy                           1.00        66
   macro avg       1.00      1.00      1.00        66
weighted avg       1.00      1.00      1.00        66

              preci